In [134]:
p_storage_dir = "../../storage"

In [135]:
import pandas as pd
import numpy as np
import re
import uuid
from pathlib import Path
from datetime import datetime

DA_STORAGE_DIR = Path(f"{p_storage_dir}/dados_abertos")

# Estágios Curriculares

In [136]:
estagios_curriculares = pd.read_csv(DA_STORAGE_DIR / "transformed/estagios_curriculares.csv", delimiter=";")

estagios_curriculares.head()

,campus,curso,modalidade,nivel,data_inicio,data_fim,data_relatorio,ofertante,status,tipo,remunerado,valor_remuneracao
0,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,Integrado,Ensino Médio,01/04/2019,30/11/2019,07/02/2020,Laboratório Ideal de Análises Clínicas Ltda Me,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN
1,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,Integrado,Ensino Médio,01/11/2017,23/01/2018,28/02/2018,Laboratório Ideal de Análises Clínicas Ltda Me,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN
2,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,Integrado,Ensino Médio,01/11/2017,15/03/2018,17/05/2018,Rafael Seiça Gonçalves de Macena,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN
3,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,Integrado,Ensino Médio,02/08/2019,30/12/2019,14/01/2020,Ali & Ramos Ltda-me,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN
4,Câmpus Águas Lindas,Técnico Integrado ao Ensino Médio em Análises ...,Integrado,Ensino Médio,03/04/2019,06/11/2019,15/01/2020,Clinvip Águas Lindas,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN


**Substituir o câmpus pela sua sigla**

In [137]:
estagios_curriculares['campus'].unique()

array(['Câmpus Águas Lindas', 'Câmpus Anápolis',
       'Câmpus Aparecida de Goiânia', 'Câmpus Cidade de Goiás',
       'Câmpus Formosa', 'Câmpus Goiânia', 'Câmpus Goiânia Oeste',
       'Câmpus Inhumas', 'Câmpus Itumbiara', 'Câmpus Jataí',
       'Câmpus Luziânia', 'Câmpus Senador Canedo', 'Câmpus Uruaçu',
       'Câmpus Valparaíso', 'Câmpus Virtual de Ead'], dtype=object)

In [138]:
campus_replacement_map = {
    "Câmpus Águas Lindas": "LIN",
    "Câmpus Anápolis": "ANA",
    "Câmpus Aparecida de Goiânia": "APA",
    "Câmpus Cidade de Goiás": "GOI",
    "Câmpus Formosa": "FOR",
    "Câmpus Goiânia": "GYN",
    "Câmpus Goiânia Oeste": "OES",
    "Câmpus Inhumas": "INH",
    "Câmpus Itumbiara": "ITU",
    "Câmpus Jataí": "JAT",
    "Câmpus Luziânia": "LUZ",
    "Câmpus Senador Canedo": "SEN",
    "Câmpus Uruaçu": "URU",
    "Câmpus Valparaíso": "VAL"
}

In [139]:
estagios_curriculares['campus'].replace(campus_replacement_map, inplace=True)

In [140]:
estagios_curriculares['campus'].unique()

array(['LIN', 'ANA', 'APA', 'GOI', 'FOR', 'GYN', 'OES', 'INH', 'ITU',
       'JAT', 'LUZ', 'SEN', 'URU', 'VAL', 'Câmpus Virtual de Ead'],
      dtype=object)

**Inserir um código único para cada registro**

In [141]:
estagios_curriculares['codigo'] = [str(uuid.uuid4()) for _ in range(len(estagios_curriculares))]

In [142]:
estagios_curriculares['codigo'].head()

0    aca13278-e1ea-41cb-a1f3-dc9c6a40c20e
1    0c6de42b-994f-4627-864e-dcc4bf887666
2    be30a467-71cc-432a-b999-0c5965d5b9ca
3    1c7a97be-3a91-47c2-a5d2-cb69f0d1920a
4    993fcfc6-ad02-4a3f-952c-47b4adb444b1
Name: codigo, dtype: object

**Remover os registros relacionados ao Campus Virtual de Ead**

Como não temos dados suficientes sobre esse campus ele será removido para evitar ruído nos dados.

In [143]:
estagios_curriculares.drop(
    estagios_curriculares[estagios_curriculares['campus'] == 'Câmpus Virtual de Ead'].index, inplace=True)

**Criar uma coluna "codigo_curso" e usar o "campus" e o "curso" para preenchê-la com o código do curso**

In [144]:
cursos = pd.read_csv(DA_STORAGE_DIR / "preprocessed/cursos.csv", delimiter=";")

In [145]:
cursos.head()

,campus,nome,codigo,modalidade,formato,turno,periodo_de_ingresso,qtd_vagas_ano,nivel,ch_disciplinas,ch_complementar,ch_estagio,ch_optativas,ch_projeto_final,ch_total,qtd_semestres
0,LIN,Técnico Integrado ao Ensino Médio em Análises ...,967,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3683,3186.0,120.0,160.0,108.0,3574.0,6
1,LIN,Técnico Integrado ao Ensino Médio em Meio Ambi...,968,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3684,2970.0,120.0,160.0,54.0,3304.0,6
2,LIN,Técnico Integrado ao Ensino Médio em Vigilânci...,908,Integrado,Presencial,Integral,Anual,30,Ensino Médio,3592,3240.0,120.0,160.0,54.0,3574.0,6
3,LIN,Técnico Integrado ao Ensino Médio em Enfermage...,909,Integrado - EJA,Presencial,Noturno,Semestral,60,Ensino Médio,3593,2547.0,120.0,0.0,0.0,2667.0,8
4,LIN,Licenciatura em Ciências Biológicas,1174,Licenciatura,Presencial,Noturno,Anual,30,Superior,4316,2960.0,200.0,0.0,108.0,3268.0,8


In [146]:
estagios_curriculares_merged = pd.merge(
    estagios_curriculares, cursos[['campus', 'nome', 'codigo']], 
    left_on=['curso', 'campus'],
    right_on=['nome', 'campus'],
    how='left'
).drop_duplicates(subset=['codigo_x'])

In [147]:
estagios_curriculares_merged.drop(columns=['nome'], inplace=True)

In [148]:
estagios_curriculares_merged.rename(columns={'codigo_x': 'codigo', 'codigo_y': 'codigo_curso', 'campus': 'sigla_campus'}, inplace=True)

In [149]:
estagios_curriculares_merged['codigo_curso'] = estagios_curriculares_merged['codigo_curso'].astype('Int64')

In [150]:
estagios_curriculares = estagios_curriculares_merged

In [151]:
estagios_curriculares.sample(n=5)

,sigla_campus,curso,modalidade,nivel,data_inicio,data_fim,data_relatorio,ofertante,status,tipo,remunerado,valor_remuneracao,codigo,codigo_curso
15561,ITU,Bacharelado em Engenharia Elétrica,Bacharelado,Superior,10/12/2020,NaN,NaN,Louis Dreyfus Company Brasil S. A.,Estágio em Andamento,Não Obrigatório,Remunerado declarando valor,820.0,493cb467-5f2d-4f6c-aa43-eb7b6b7d8300,441
10105,GYN,Superior de Tecnologia em Saneamento Ambiental,Tecnólogo,Superior,22/11/2010,08/03/2011,25/04/2011,Dbo Engenharia Ltda,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN,c8a4443f-798c-467e-93d9-5a0fc2cd4b7e,<NA>
14211,OES,Técnico Integrado ao Ensino Médio em Nutrição ...,Integrado,Ensino Médio,05/04/2017,19/01/2018,20/01/2018,Qualitá Refeições,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN,10b1abdb-8604-44c1-bfe1-6651eb8502b2,969
17923,JAT,Técnico Subsequente ao Ensino Médio em Açucar ...,Subsequente,Ensino Médio,08/05/2009,17/04/2015,17/03/2015,Energética Serranópolis Ltda.,Estágio Encerrado e virou Emprego,Obrigatório,Remunerado a combinar,NaN,e12873e2-4045-4497-82c4-ce55a12bb3cb,<NA>
5077,GYN,Bacharelado em Engenharia Civil,Bacharelado,Superior,01/09/2020,22/03/2021,NaN,Detran - Go - Departamento Estadual de Trânsit...,Estágio Encerrado e Não virou Emprego,Não Obrigatório,Remunerado declarando valor,1000.0,a3743dd4-5c32-4aa6-a0a8-3b15c2bbf470,599


**Normalizar as datas, colocando-as no formato YYYY-mm-dd**

In [152]:
def convert_date_from_br_to_iso(date_br_format: str):
    if pd.isna(date_br_format): return
    
    date_obj = datetime.strptime(date_br_format, '%d/%m/%Y')
    date_iso_format = date_obj.strftime('%Y-%m-%d')
    return date_iso_format

In [153]:
estagios_curriculares['data_inicio'] = estagios_curriculares['data_inicio'].apply(convert_date_from_br_to_iso)
estagios_curriculares['data_fim'] = estagios_curriculares['data_fim'].apply(convert_date_from_br_to_iso)
estagios_curriculares['data_relatorio'] = estagios_curriculares['data_relatorio'].apply(convert_date_from_br_to_iso)

In [154]:
estagios_curriculares.sample(n=5)

,sigla_campus,curso,modalidade,nivel,data_inicio,data_fim,data_relatorio,ofertante,status,tipo,remunerado,valor_remuneracao,codigo,codigo_curso
20831,URU,Técnico Integrado em Manutenção e Suporte em I...,Integrado,Ensino Médio,2014-03-18,2014-08-18,2014-08-19,Acepeças - Comércio de Acessórios e Peças para...,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN,085382c1-7f6d-44a9-aa24-6d2c7bd6b71a,<NA>
12032,GYN,Técnico Integrado ao Ensino Médio em Informáti...,Integrado - EJA,Ensino Médio,2017-09-18,2017-11-30,2017-12-19,Agência Brasil Central,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN,eaee97fe-ddcf-4ee1-aeae-cfcd35cd63dc,1017
4929,GYN,Bacharelado em Engenharia Cartográfica e de Ag...,Bacharelado,Superior,2020-12-01,2021-05-31,2021-05-31,Cpe Comercio de Equipamentos Topográficos Eireli,Estágio Encerrado e não há informações sobre e...,Obrigatório,Remunerado declarando valor,800.0,cc29d52a-8c4e-4c25-a43b-573ccb456c5f,983
8990,GYN,Superior de Tecnologia em Planejamento de Tran...,Tecnólogo,Superior,2008-01-02,2008-04-30,2008-04-30,Inspecentro- Inspeção de Veículos e Equipament...,Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN,7361e7f5-8976-484e-a36f-a9faee1bdf86,<NA>
8024,GYN,Superior de Tecnologia em Construção de Vias T...,Tecnólogo,Superior,2014-03-17,2015-03-17,2015-03-18,Novageo do Brasil Ltda.,Estágio Encerrado e virou Emprego,Obrigatório,Remunerado a combinar,NaN,9450c918-9201-43de-9e60-7173f2c5e883,<NA>


**Limpar o caracteres indesejados nos dados da coluna com o nome do ofertante do estágio**

In [155]:
estagios_curriculares[estagios_curriculares['ofertante'].str.contains('"')]

,sigla_campus,curso,modalidade,nivel,data_inicio,data_fim,data_relatorio,ofertante,status,tipo,remunerado,valor_remuneracao,codigo,codigo_curso
12501,GYN,Técnico Integrado ao Ensino Médio em Serviços ...,Integrado - EJA,Ensino Médio,2010-08-01,2010-12-01,2010-12-06,"Escola Municipal ""o Pequenino""",Estágio Encerrado e não há informações sobre e...,Obrigatório,Não Remunerado,NaN,36555bc7-b70d-4bfd-8290-5cab1760a0eb,38


In [156]:
def clear_double_quotes(text):
    if isinstance(text, str):
        return text.replace('"', '')

In [157]:
estagios_curriculares['ofertante'] = estagios_curriculares['ofertante'].apply(clear_double_quotes)

**Exportar o dataset**

In [158]:
export_path = Path(DA_STORAGE_DIR / "preprocessed")

if not export_path.is_dir():
    Path(export_path).mkdir()

try:
    estagios_curriculares.to_csv(export_path / "estagios_curriculares.csv", sep=";", index=False)
except Exception as e:
    print(str(e))